# Cleaning and analyzing - contract processes
This notebook will be used for cleaning and analyzing the data scraped from [Sercop's search page](https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/buscarProceso.cpe?sg=1#), scraped with the previuos notebook. 

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("compras_medicinas_2022-2023.csv")

In [3]:
df.head(1)

,codigo,detalle_url,entidad,objeto,estado,ubicacion,presupuesto,fecha_publicacion,semestre
0,SICM-499-2022,https://modulocomprascorporativas.compraspubli...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,DCI: LEVONORGESTREL - FORMA FARMACÉUTICA: SÓLI...,Desierto,PICHINCHA / QUITO,$62.31600,2022-06-23 08:00:00,First 2022


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 986 entries, 0 to 985
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   codigo             986 non-null    object
 1   detalle_url        986 non-null    object
 2   entidad            986 non-null    object
 3   objeto             986 non-null    object
 4   estado             986 non-null    object
 5   ubicacion          986 non-null    object
 6   presupuesto        986 non-null    object
 7   fecha_publicacion  986 non-null    object
 8   semestre           986 non-null    object
dtypes: object(9)
memory usage: 69.5+ KB


In [5]:
df = df.drop(columns="semestre")

In [6]:
df[df["objeto"].str.contains('"')]

,codigo,detalle_url,entidad,objeto,estado,ubicacion,presupuesto,fecha_publicacion
826,SICM2-534-2022,https://modulocomprascorporativas.compraspubli...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,"""DCI: HIERRO, MULTIVITAMINAS Y MINERALES: HIER...",Desierto,PICHINCHA / QUITO,$0.04104,2023-10-06 08:00:00
901,SICM2-345-2022,https://modulocomprascorporativas.compraspubli...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,"\""DCI: COMPLEJO B: TIAMINA VITAMINA B1 - PIRID...",Verificación de documentación habilitante otro...,PICHINCHA / QUITO,$0.15161,2023-08-29 08:00:00
911,SICM2-475-2022,https://modulocomprascorporativas.compraspubli...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,"\""DCI: COMPLEJO B: TIAMINA VITAMINA B1 - PIRID...",Verificación de documentación habilitante otro...,PICHINCHA / QUITO,$0.98436,2023-08-29 08:00:00


In [7]:
df["objeto"] = (
    df["objeto"].str.replace("\\", "", regex=False)
        .str.replace('"', '')
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
)

In [8]:
df["provincia"] = df["ubicacion"].str.split("/").str[0].str.title()
df["canton"] = df["ubicacion"].str.split("/").str[1].str.title()

In [9]:
df = df.drop(columns="ubicacion")

In [10]:
df["presupuesto"] = df["presupuesto"].str.replace("$", "").str.replace(",", "").astype(float)

In [11]:
df["principio_activo"] = df["objeto"].str.split(": ").str[1].str.split(" -").str[0].str.title()
df["forma_farmaceutica"] = df["objeto"].str.split("TICA: ").str[1].str.split(" - CON").str[0].str.title()
df["concentracion"] = df["objeto"].str.split("TRACIÓN: ").str[1].str.split(" - PRE").str[0]
df["presentacion_comercial"] = df["objeto"].str.split("CIAL: ").str[1].str.title()

In [12]:
df = df.rename(columns={"objeto": "item"})

In [13]:
df.columns

Index(['codigo', 'detalle_url', 'entidad', 'item', 'estado', 'presupuesto',
       'fecha_publicacion', 'provincia', 'canton', 'principio_activo',
       'forma_farmaceutica', 'concentracion', 'presentacion_comercial'],
      dtype='object')

In [14]:
df = df[['codigo', 'detalle_url', 'item', 'principio_activo', 'forma_farmaceutica', 'concentracion',
         'presentacion_comercial', 'estado', 'presupuesto', 'provincia', 'canton', 'fecha_publicacion']]

In [15]:
df.head(1)

,codigo,detalle_url,item,principio_activo,forma_farmaceutica,concentracion,presentacion_comercial,estado,presupuesto,provincia,canton,fecha_publicacion
0,SICM-499-2022,https://modulocomprascorporativas.compraspubli...,DCI: LEVONORGESTREL - FORMA FARMACÉUTICA: SÓLI...,Levonorgestrel,Sólido Parenteral -Implante Subdérmico,150 MG -2 VARILLAS DE 75 MG,Caja X Implante -S- De 75 Mg C/U + Trocar -Es.,Desierto,62.316,Pichincha,Quito,2022-06-23 08:00:00


In [16]:
df.to_csv("cleaned_processes.csv", index=False)

## Analysis

For this analysis, it is important to distinguish between the different variables:
- The "item" variable refers to the object of each procurement process. It includes a unique combination of the active ingredient, pharmaceutical form, concentration, and commercial presentation.
- The variables 'principio_activo', 'forma_farmaceutica', 'concentracion','presentacion_comercial' are derived from "item", but will be used separately for the purposes of the analysis below.

In [17]:
#How many purchsings processes have been launched since 2022?
len(df)

986

In [18]:
#How many items have had at least one process launched
df["item"].nunique()

830

In [19]:
# How many different active principles had at least one process launched
df["principio_activo"].nunique()

380

In [20]:
#Which medicines have more than 1 process launched?
df["item"].value_counts().head().to_frame()

,count
item,
DCI: MISOPROSTOL - FORMA FARMACÉUTICA: SÓLIDO ORAL - CONCENTRACIÓN: 200 MCG - PRESENTACIÓN COMERCIAL: CAJA X BLÍSTER/RISTRA,3
"DCI: FENITOÍNA - FORMA FARMACÉUTICA: SÓLIDO ORAL - CONCENTRACIÓN: 100 MG - PRESENTACIÓN COMERCIAL: CAJA X BLÍSTER/RISTRA/FRASCO DOSIS PERSONAL, NO ENVASE HOSPITALARIO",3
DCI: OSELTAMIVIR - FORMA FARMACÉUTICA: SÓLIDO ORAL - CONCENTRACIÓN: 75 MG - PRESENTACIÓN COMERCIAL: CAJA X BLÍSTER/RISTRA,2
DCI: DEXAMETASONA - FORMA FARMACÉUTICA: LÍQUIDO OFTÁLMICO - CONCENTRACIÓN: 1 MG/ML - PRESENTACIÓN COMERCIAL: CAJA X FRASCO GOTERO X 5 ML -MÍNIMO-,2
DCI: BIPERIDENO - FORMA FARMACÉUTICA: SÓLIDO ORAL - CONCENTRACIÓN: 4 MG - PRESENTACIÓN COMERCIAL: CAJA X BLÍSTER/RISTRA,2


In [21]:
#Check the processes for Misoprostol launched 3 times
df[df["item"] == "DCI: MISOPROSTOL - FORMA FARMACÉUTICA: SÓLIDO ORAL - CONCENTRACIÓN: 200 MCG - PRESENTACIÓN COMERCIAL: CAJA X BLÍSTER/RISTRA"]

,codigo,detalle_url,item,principio_activo,forma_farmaceutica,concentracion,presentacion_comercial,estado,presupuesto,provincia,canton,fecha_publicacion
147,SICM-391-2022,https://modulocomprascorporativas.compraspubli...,DCI: MISOPROSTOL - FORMA FARMACÉUTICA: SÓLIDO ...,Misoprostol,Sólido Oral,200 MCG,Caja X Blíster/Ristra,Desierto,0.42000,Pichincha,Quito,2022-05-27 08:00:00
899,SICM2-391-2022-A,https://modulocomprascorporativas.compraspubli...,DCI: MISOPROSTOL - FORMA FARMACÉUTICA: SÓLIDO ...,Misoprostol,Sólido Oral,200 MCG,Caja X Blíster/Ristra,Verificación de documentación habilitante otro...,0.43094,Pichincha,Quito,2023-08-29 08:00:00
972,SICM2-391-2022,https://modulocomprascorporativas.compraspubli...,DCI: MISOPROSTOL - FORMA FARMACÉUTICA: SÓLIDO ...,Misoprostol,Sólido Oral,200 MCG,Caja X Blíster/Ristra,Cancelado,0.04309,Pichincha,Quito,2023-07-21 08:00:00


In [22]:
#Check the processes for Fenitoína launched 3 times
df[df["item"] == "DCI: FENITOÍNA - FORMA FARMACÉUTICA: SÓLIDO ORAL - CONCENTRACIÓN: 100 MG - PRESENTACIÓN COMERCIAL: CAJA X BLÍSTER/RISTRA/FRASCO DOSIS PERSONAL, NO ENVASE HOSPITALARIO"]

,codigo,detalle_url,item,principio_activo,forma_farmaceutica,concentracion,presentacion_comercial,estado,presupuesto,provincia,canton,fecha_publicacion
651,SICM-569-2022,https://modulocomprascorporativas.compraspubli...,DCI: FENITOÍNA - FORMA FARMACÉUTICA: SÓLIDO OR...,Fenitoína,Sólido Oral,100 MG,"Caja X Blíster/Ristra/Frasco Dosis Personal, N...",Desierto,0.04500,Pichincha,Quito,2022-08-09 08:00:00
728,SICM-569-2022-A,https://modulocomprascorporativas.compraspubli...,DCI: FENITOÍNA - FORMA FARMACÉUTICA: SÓLIDO OR...,Fenitoína,Sólido Oral,100 MG,"Caja X Blíster/Ristra/Frasco Dosis Personal, N...",Desierto,0.04500,Pichincha,Quito,2023-02-24 08:00:00
824,SICM2-569-2022,https://modulocomprascorporativas.compraspubli...,DCI: FENITOÍNA - FORMA FARMACÉUTICA: SÓLIDO OR...,Fenitoína,Sólido Oral,100 MG,"Caja X Blíster/Ristra/Frasco Dosis Personal, N...",Verificación de documentación habilitante otro...,0.04617,Pichincha,Quito,2023-10-06 08:00:00


In [23]:
# Which kind of status of processes
df["estado"].value_counts().to_frame()

,count
estado,
Desierto,335
Adjudicado oferente ganador,274
Adjudicado otros oferentes,140
Verificación de documentación habilitante otros oferentes,137
Verificación de documentación habilitante oferente ganador,52
Por Adjudicar oferente ganador,22
Por Adjudicar otros oferentes,18
Suspendido,7
Cancelado,1


In [95]:
# Awarded contracts
len(df[df["estado"].str.contains("Adjudicado")])

414

In [96]:
# Failed contracts
len(df[~df["estado"].str.contains("Adjudicado")])

572

### Medicine that never got a contract
From the previous analysis, we can see there are items that never got a contract, even if they have several purchasing processes launched.

In [24]:
#Items that have an assigned supplier (adjudicado)
adjudicados = df.groupby("item")["estado"].apply(lambda x: x.isin(["Adjudicado oferente ganador", "Adjudicado otros oferentes"]).any())
adjudicados.value_counts()

estado
False    416
True     414
Name: count, dtype: int64

In [25]:
# Items that never got a contract
never_adjudicados = adjudicados[~adjudicados].index.to_list()
df_nocontract = pd.DataFrame(never_adjudicados, columns=["item"])

In [26]:
len(df_nocontract)

416

In [27]:
df_nocontract.head()

,item
0,DCI: ACICLOVIR - FORMA FARMACÉUTICA: LÍQUIDO O...
1,DCI: ACICLOVIR - FORMA FARMACÉUTICA: SEMISÓLID...
2,DCI: ACICLOVIR - FORMA FARMACÉUTICA: SÓLIDO PA...
3,DCI: ACICLOVIR - FORMA FARMACÉUTICA:LÍQUIDO OR...
4,DCI: ACICLOVIR - FORMA FARMACÉUTICA:SÓLIDO ORA...


In [28]:
df_nocontract["principio activo"] = df_nocontract["item"].str.split("I: ").str[1].str.split(" -").str[0].str.title()
df_nocontract["forma_farmaceutica"] = df_nocontract["item"].str.split("TICA: ").str[1].str.split(" - CON").str[0].str.title()
df_nocontract["concentracion"] = df_nocontract["item"].str.split("TRACIÓN: ").str[1].str.split(" - PRE").str[0]
df_nocontract["presentacion_comercial"] = df_nocontract["item"].str.split("CIAL: ").str[1]
df_nocontract.head(2)

,item,principio activo,forma_farmaceutica,concentracion,presentacion_comercial
0,DCI: ACICLOVIR - FORMA FARMACÉUTICA: LÍQUIDO O...,Aciclovir,Líquido Oral,200 MG/5 ML,CAJA X FRASCO X 100 ML -MÍNIMO- CON DOSIFICADOR
1,DCI: ACICLOVIR - FORMA FARMACÉUTICA: SEMISÓLID...,Aciclovir,Semisólido Oftálmico,3 %,CAJA X TUBO X 5 G


In [29]:
# Active principles that have failed processes the most
df_nocontract["principio activo"].value_counts().head(10).to_frame()

,count
principio activo,
Metronidazol,8
Morfina,8
Levotiroxina Sódica,7
Oxicodona,7
Leuprolide,6
Aciclovir,5
Lidocaína Con Epinefrina,5
Clonazepam,5
Prednisolona,5


In [30]:
#Example - what kinds of morfine didn't get a contract
df_nocontract[df_nocontract["item"].str.contains("MORFINA")]

,item,principio activo,forma_farmaceutica,concentracion,presentacion_comercial
293,DCI: MORFINA - FORMA FARMACÉUTICA: LÍQUIDO ORA...,Morfina,Líquido Oral,2 MG/ML,CAJA X FRASCO X 60 ML MÍNIMO - CON DOSIFICADOR
294,DCI: MORFINA - FORMA FARMACÉUTICA: LÍQUIDO ORA...,Morfina,Líquido Oral,2 MG/ML,CAJA X FRASCO X 60 ML MÍNIMO- CON DOSIFICADOR
295,DCI: MORFINA - FORMA FARMACÉUTICA: LÍQUIDO ORA...,Morfina,Líquido Oral,20 MG/ML,CAJA X FRASCO X 60 ML MÍNIMO - CON DOSIFICADOR
296,DCI: MORFINA - FORMA FARMACÉUTICA: LÍQUIDO ORA...,Morfina,Líquido Oral,20 MG/ML,CAJA X FRASCO X 60 ML MÍNIMO CON DOSIFICADOR
297,DCI: MORFINA - FORMA FARMACÉUTICA: SÓLIDO ORAL...,Morfina,Sólido Oral -Liberación Prolongada-,30 MG,CAJA X BLÍSTER/RISTRA
298,DCI: MORFINA - FORMA FARMACÉUTICA: SÓLIDO ORAL...,Morfina,Sólido Oral Liberación Prolongada,10 MG,CAJA X BLÍSTER/RISTRA
299,DCI: MORFINA - FORMA FARMACÉUTICA: SÓLIDO ORAL...,Morfina,Sólido Oral Liberación Prolongada,30 MG,CAJA X BLÍSTER/RISTRA
300,DCI: MORFINA - FORMA FARMACÉUTICA: SÓLIDO ORAL...,Morfina,Sólido Oral Liberación Prolongada,60 MG,CAJA X BLÍSTER/RISTRA


In [31]:
# Is there morfine that DID get a contract?
df[(df["principio_activo"] == "Morfina") & (df["estado"].str.contains("Adjudicado"))]

,codigo,detalle_url,item,principio_activo,forma_farmaceutica,concentracion,presentacion_comercial,estado,presupuesto,provincia,canton,fecha_publicacion
401,SICM-129-2022,https://modulocomprascorporativas.compraspubli...,DCI: MORFINA - FORMA FARMACÉUTICA: LÍQUIDO PAR...,Morfina,Líquido Parenteral,10 MG/ML,Caja X Ampolla - S X 1 Ml,Adjudicado oferente ganador,0.73013,Pichincha,Quito,2022-03-29 08:00:00
560,SICM-658-2022,https://modulocomprascorporativas.compraspubli...,DCI: MORFINA - FORMA FARMACÉUTICA: LÍQUIDO PAR...,Morfina,Líquido Parenteral,20 MG/ML,Caja X Ampolla-S- X 1 Ml,Adjudicado oferente ganador,0.85964,Pichincha,Quito,2022-08-15 08:00:00
720,SICM-441-2022-A,https://modulocomprascorporativas.compraspubli...,DCI: MORFINA - FORMA FARMACÉUTICA: SÓLIDO ORAL...,Morfina,Sólido Oral,10 MG,Caja X Blíster/Ristra,Adjudicado oferente ganador,0.44587,Pichincha,Quito,2023-02-24 08:00:00


In [97]:
df[(df["principio_activo"] == "Metronidazol") & (df["estado"].str.contains("Adjudicado"))]

,codigo,detalle_url,item,principio_activo,forma_farmaceutica,concentracion,presentacion_comercial,estado,presupuesto,provincia,canton,fecha_publicacion
293,SICM-237-2022,https://modulocomprascorporativas.compraspubli...,DCI: METRONIDAZOL - FORMA FARMACÉUTICA: SÓLIDO...,Metronidazol,Sólido Vaginal,500 MG,Caja X Blíster/Ristra,Adjudicado oferente ganador,0.125,Pichincha,Quito,2022-05-03 08:00:00
509,SICM-007-2022,https://modulocomprascorporativas.compraspubli...,DCI: METRONIDAZOL - FORMA FARMACÉUTICA: LÍQUID...,Metronidazol,Líquido Parenteral,5 MG/ML,Frasco Plástico/Funda Autocolapsable X 100 Ml,Adjudicado oferente ganador,0.500,Pichincha,Quito,2022-03-23 08:00:00


In [32]:
df_nocontract.columns

Index(['item', 'principio activo', 'forma_farmaceutica', 'concentracion',
       'presentacion_comercial'],
      dtype='object')

In [33]:
# Is there active principles that, in any form, didn't get any contract?
adjudicados_principles = df.groupby("principio_activo")["estado"].apply(lambda x: x.isin(["Adjudicado oferente ganador", "Adjudicado otros oferentes"]).any())
adjudicados_principles.value_counts()

estado
True     264
False    116
Name: count, dtype: int64

In [34]:
never_adjudicados_principles = adjudicados_principles[~adjudicados_principles].index.to_list()
len(never_adjudicados_principles)

116

In [35]:
never_adjudicados_principles

['Agua Para Irrigación',
 'Alopurinol',
 'Alquitrán De Hulla',
 'Amitriptilina',
 'Ampicilina + Sulbactam',
 'Asparaginasa',
 'Bencilpenicilina Benzatínica Penicilina G Benzatínica',
 'Bencilpenicilina Penicilina G Cristalina',
 'Bendamustina',
 'Bicarbonato De Sodio',
 'Bupivacaína',
 'Bupivacaína Sin Epinefrina',
 'Cafeína',
 'Calcipotriol + Betametasona Dipropionato',
 'Carbón Activado',
 'Cefepima',
 'Clomifeno',
 'Clonazepam',
 'Colistina',
 'Combinaciones Lactato De Ringer',
 'Darunavir',
 'Daunorubicina',
 'Difenhidramina',
 'Dimenhidrinato',
 'Dolutegravir',
 'Dopamina',
 'Doxazosina',
 'Dutasterida',
 'Efedrina',
 'Epirubicina',
 'Espiramicina',
 'Estreptoquinasa',
 'Etravirina',
 'Fenobarbital',
 'Ferroso Sulfato',
 'Fludarabina',
 'Folinato Cálcico Leucovorina',
 'Gemfibrozilo',
 'Gliclazida',
 'Glucagón',
 'Griseofulvina',
 'Hidralazina',
 'Hidroxocobalamina',
 'Hierro, Multivitaminas Y Minerales',
 'Ifosfamida',
 'Imatinib',
 'Insulina Humana',
 'Insulina Humana Acción Ráp

In [36]:
df_nocontract["tiene_otro_contrato"] = df_nocontract["principio activo"].apply(lambda x: "No" if x in never_adjudicados_principles else "Si")

In [37]:
df_nocontract.to_csv("sin_contrato.csv", index=False)

### Items that got a contract

In [38]:
procesos_adjudicados = df[(df["estado"] == "Adjudicado oferente ganador") | (df["estado"] == "Adjudicado otros oferentes")]
len(procesos_adjudicados)

414

In [39]:
procesos_adjudicados.head(2)

,codigo,detalle_url,item,principio_activo,forma_farmaceutica,concentracion,presentacion_comercial,estado,presupuesto,provincia,canton,fecha_publicacion
2,SICM-500-2022,https://modulocomprascorporativas.compraspubli...,DCI: LIDOCAÍNA SIN EPINEFRINA - FORMA FARMACÉU...,Lidocaína Sin Epinefrina,Sólido Cutáneo Parche Transdérmico,5 %,Caja X Parche/Parches,Adjudicado oferente ganador,5.00,Pichincha,Quito,2022-06-23 08:00:00
9,SICM-519-2022,https://modulocomprascorporativas.compraspubli...,DCI: LOPINAVIR + RITONAVIR - FORMA FARMACÉUTIC...,Lopinavir + Ritonavir,Sólido Oral,200 MG + 50 MG,"Caja X Frasco Dosis Personal, No Envase Hospit...",Adjudicado otros oferentes,0.27,Pichincha,Quito,2022-06-23 08:00:00


In [40]:
procesos_adjudicados["principio_activo"].nunique()

264

In [41]:
procesos_adjudicados["item"].nunique()

414

In [92]:
procesos_adjudicados["principio_activo"].value_counts().to_frame().head(12)

,count
principio_activo,
Paracetamol,8
Risperidona,5
Levotiroxina Sódica,5
Cloruro De Sodio,5
Quetiapina,4
Eritropoyetina,4
Clindamicina,4
Metformina,4
Enoxaparina,4


In [89]:
procesos_adjudicados[procesos_adjudicados["principio_activo"] == "Paracetamol"]

,codigo,detalle_url,item,principio_activo,forma_farmaceutica,concentracion,presentacion_comercial,estado,presupuesto,provincia,canton,fecha_publicacion
378,SICM-136-2022,https://modulocomprascorporativas.compraspubli...,DCI: PARACETAMOL - FORMA FARMACÉUTICA: LÍQUIDO...,Paracetamol,Líquido Oral,160 MG/5 ML,Caja X Frasco X 60 Ml - Mínimo - Con Dosificador,Adjudicado oferente ganador,0.61005,Pichincha,Quito,2022-03-29 08:00:00
379,SICM-137-2022,https://modulocomprascorporativas.compraspubli...,DCI: PARACETAMOL - FORMA FARMACÉUTICA: LÍQUIDO...,Paracetamol,Líquido Parenteral,10 MG/ML,Caja X Vial-Es X 50 Ml,Adjudicado oferente ganador,1.00000,Pichincha,Quito,2022-03-29 08:00:00
380,SICM-138-2022,https://modulocomprascorporativas.compraspubli...,DCI: PARACETAMOL - FORMA FARMACÉUTICA: LÍQUIDO...,Paracetamol,Líquido Parenteral,10 MG/ML,Caja X Vial-Es X 100 Ml,Adjudicado otros oferentes,0.69000,Pichincha,Quito,2022-03-29 08:00:00
407,SICM-135-2022,https://modulocomprascorporativas.compraspubli...,DCI: PARACETAMOL - FORMA FARMACÉUTICA: LÍQUIDO...,Paracetamol,Líquido Oral,150 MG/5 ML,Caja X Frasco X 60 Ml - Mínimo Con Dosificador,Adjudicado oferente ganador,0.70550,Pichincha,Quito,2022-03-29 08:00:00
475,SICM-051-2022,https://modulocomprascorporativas.compraspubli...,DCI: PARACETAMOL - FORMA FARMACÉUTICA: LÍQUIDO...,Paracetamol,Líquido Oral,100 MG/ML,Caja X Frasco Gotero X 15 Ml -Mínimo-,Adjudicado oferente ganador,0.24246,Pichincha,Quito,2022-03-24 08:00:00
518,SICM-024-2022,https://modulocomprascorporativas.compraspubli...,DCI: PARACETAMOL - FORMA FARMACÉUTICA: SÓLIDO ...,Paracetamol,Sólido Oral,500 MG,Caja X Blíster/Ristra,Adjudicado oferente ganador,0.00683,Pichincha,Quito,2022-03-23 08:00:00
942,SICM2-134-2022,https://modulocomprascorporativas.compraspubli...,DCI: PARACETAMOL - FORMA FARMACÉUTICA: LÍQUIDO...,Paracetamol,Líquido Oral,120 MG/5 ML,Caja X Frasco X 60 Ml Mínimo- Con Dosificador,Adjudicado oferente ganador,0.51302,Pichincha,Quito,2023-07-26 08:00:00
956,SICM2-139-2022,https://modulocomprascorporativas.compraspubli...,DCI: PARACETAMOL - FORMA FARMACÉUTICA: SÓLIDO ...,Paracetamol,Sólido Rectal,300 MG,Caja X Blíster/Ristra,Adjudicado oferente ganador,0.17135,Pichincha,Quito,2023-07-21 08:00:00


In [58]:
procesos_adjudicados["principio_activo"].value_counts().to_frame().tail(10)

,count
principio_activo,
Azitromicina,1
Claritromicina,1
Ácido Fusídico,1
Levodopa + Carbidopa,1
Levomepromazina,1
Diazepam,1
Alprazolam,1
Dexmedetomidina,1
Neostigmina,1


In [76]:
procesos_adjudicados.sort_values(by="presupuesto", ascending=False).head(10)

,codigo,detalle_url,item,principio_activo,forma_farmaceutica,concentracion,presentacion_comercial,estado,presupuesto,provincia,canton,fecha_publicacion
34,SICM-472-2022,https://modulocomprascorporativas.compraspubli...,DCI: CETUXIMAB - FORMA FARMACÉUTICA: LÍQUIDO P...,Cetuximab,Líquido Parenteral,5 MG/ML,Caja X Vial-Es X 100 Ml,Adjudicado oferente ganador,1817.74727,Pichincha,Quito,2022-06-22 08:00:00
554,SICM-652-2022,https://modulocomprascorporativas.compraspubli...,DCI: BASILIXIMAB - FORMA FARMACÉUTICA: SÓLIDO ...,Basiliximab,Sólido Parenteral,20 MG,Caja X Vial-Es- Con Diluyente - Caja X Vial-Es-,Adjudicado oferente ganador,1334.44900,Pichincha,Quito,2022-08-15 08:00:00
80,SICM-444-2022,https://modulocomprascorporativas.compraspubli...,DCI: OCTREOTIDA - FORMA FARMACÉUTICA: SÓLIDO P...,Octreotida,Sólido Parenteral,30 MG,Caja X Vial Con Diluyente,Adjudicado otros oferentes,1200.00000,Pichincha,Quito,2022-06-21 08:00:00
377,SICM-168-2022,https://modulocomprascorporativas.compraspubli...,DCI: TRASTUZUMAB - FORMA FARMACÉUTICA: SÓLIDO ...,Trastuzumab,Sólido Parenteral,440 MG,Caja X Vial Con Diluyente,Adjudicado oferente ganador,875.00000,Pichincha,Quito,2022-03-30 08:00:00
418,SICM-113-2022,https://modulocomprascorporativas.compraspubli...,DCI: OCTREOTIDA - FORMA FARMACÉUTICA: SÓLIDO P...,Octreotida,Sólido Parenteral,20 MG,Caja X Vial Con Diluyente,Adjudicado oferente ganador,550.00000,Pichincha,Quito,2022-03-28 08:00:00
73,SICM-467-2022,https://modulocomprascorporativas.compraspubli...,DCI: ALTEPLASA - FORMA FARMACÉUTICA: SÓLIDO PA...,Alteplasa,Sólido Parenteral,50 MG,Caja X Vial-Es- Con Disolvente,Adjudicado oferente ganador,549.43158,Pichincha,Quito,2022-06-21 08:00:00
56,SICM-495-2022,https://modulocomprascorporativas.compraspubli...,DCI: INMUNOGLOBULINA ANTITIMOCÍTICA EQUINA - F...,Inmunoglobulina Antitimocítica Equina,Líquido Parenteral,50 MG/ML,Caja X Vial-Es X 5 Ml,Adjudicado oferente ganador,545.63000,Pichincha,Quito,2022-06-22 08:00:00
694,SICM-552-2022,https://modulocomprascorporativas.compraspubli...,DCI: INMUNOGLOBULINA HUMANA NORMAL PARA ADMINI...,Inmunoglobulina Humana Normal Para Administrac...,NaN,NaN,Caja X Vial-Es- X 100 Ml,Adjudicado otros oferentes,540.30890,Pichincha,Quito,2022-08-08 08:00:00
462,SICM-085-2022,https://modulocomprascorporativas.compraspubli...,DCI: BEVACIZUMAB - FORMA FARMACÉUTICA: LÍQUIDO...,Bevacizumab,Líquido Parenteral,25 MG/ML,Caja X Vial X 16 Ml,Adjudicado otros oferentes,480.00000,Pichincha,Quito,2022-03-25 08:00:00
468,SICM-046-2022,https://modulocomprascorporativas.compraspubli...,DCI: FACTOR VIII - FORMA FARMACÉUTICA: SÓLIDO ...,Factor Viii,Sólido Parenteral,1 000 UI -RECOMBINANTE- PRESENTACIÓN COMERCIAL...,Caja X Vial Con Diluyente Y Kit Para Administr...,Adjudicado oferente ganador,391.00000,Pichincha,Quito,2022-03-24 08:00:00


In [42]:
#Export so we can scrape the contract information
procesos_adjudicados.to_csv("adjudicados.csv", index=False)